# The lack of PFS: a danger to privacy

With TLS 1.2 and earlier, some cipher suites do not provide Perfect Forward Secrecy. Without this property, an attacker compromising the server private key can easily decrypt TLS traffic.

In the following example, Scapy is used to decrypt a comunication made without PFS using the ciphersuite `TLS_RSA_WITH_AES_128_CBC_SHA`, giving the server private key stored in `raw_data/pki/srv_key.pem`.

In [ ]:
from scapy.all import *
load_layer('tls')

In [ ]:
record1_str = open('raw_data/tls_session_compromised/01_cli.raw', 'rb').read()
record1 = TLS(record1_str)
record1.msg[0].show()

In [ ]:
record2_str = open('raw_data/tls_session_compromised/02_srv.raw', 'rb').read()
record2 = TLS(record2_str, tls_session=record1.tls_session.mirror())
record2.msg[0].show()

In [ ]:
# Supposing that the private key of the server was stolen,
# the traffic can be decoded by registering it to the Scapy TLS session
key = PrivKey('raw_data/pki/srv_key.pem')
record2.tls_session.server_rsa_key = key

In [ ]:
record3_str = open('raw_data/tls_session_compromised/03_cli.raw', 'rb').read()
record3 = TLS(record3_str, tls_session=record2.tls_session.mirror())
record3.show()

In [ ]:
record4_str = open('raw_data/tls_session_compromised/04_srv.raw', 'rb').read()
record4 = TLS(record4_str, tls_session=record3.tls_session.mirror())
record4.show()

In [ ]:
# This is the first TLS Record containing user data. If decryption works,
# you should see the string "To boldly go where no man has gone before..." in plaintext.
record5_str = open('raw_data/tls_session_compromised/05_cli.raw', 'rb').read()
record5 = TLS(record5_str, tls_session=record4.tls_session.mirror())
record5.show()

# Decrypting TLS Traffic Protected with PFS

When PFS is in action, the only way to break TLS 1.2 is to possess decryption keys. They can be retrieved by dumping the process memory, or making the TLS library to write then into a [NSS Key Log](https://developer.mozilla.org/en-US/docs/Mozilla/Projects/NSS/Key_Log_Format) (as allowed by OpenSSL, Chrome or Firefox).

The data used in the following examples was retrieved the following commands:
```
cd doc/notebooks/tls/raw_data/

# Start a TLS 1.12 Server using the s_server
sudo openssl s_server -accept localhost:443 -cert pki/srv_cert.pem -key pki/srv_key.pem -WWW -tls1_2

# Sniff the network and write packets to a file
sudo tcpdump -i lo -w tls_nss_example.pcap port 443

# Connect to the server using s_client and retrieve the secrets.txt file
openssl s_client -connect localhost:443 -keylogfile tls_nss_example.keys.txt
```

## Decrypt a PCAP files

Scapy can parse NSS Key logs, and use the cryptographic material to decrypt TLS traffic from a pcap file.

In [ ]:
load_layer("tls")

conf.tls_session_enable = True
conf.tls_nss_filename = "raw_data/tls_nss_example.keys.txt"

packets = rdpcap("raw_data/tls_nss_example.pcap")

In [ ]:
# Display the HTTP GET query
packets[11][TLS].show()

In [ ]:
# Display the answer containing the secret
packets[13][TLS].show()

## Decrypt Manually

Internally, the `conf.tls_session_enable` parameter makes Scapy follows TCP records, such as Client Hello or Server Hello, and updates `tlsSession` objects.

Scapy inner behavior is illustrated by the following example.

In [ ]:
# Read packets from a pcap
load_layer("tls")

packets = rdpcap("raw_data/tls_nss_example.pcap")

# Load the keys from a NSS Key Log
nss_keys = load_nss_keys("raw_data/tls_nss_example.keys.txt")

In [ ]:
# Parse the Client Hello message from its raw bytes. This configures a new tlsSession object
client_hello = TLS(raw(packets[3][TLS]))

# Parse the Server Hello message, using the mirrored client_hello tlsSession object
server_hello = TLS(raw(packets[5][TLS]), tls_session=client_hello.tls_session.mirror())

# Configure the TLS master secret retrieved from the NSS Key Log
server_hello.tls_session.master_secret = nss_keys["CLIENT_RANDOM"]["Secret"]

# Parse remaining TLS messages
client_finished = TLS(raw(packets[7][TLS]), tls_session=server_hello.tls_session.mirror())
server_finished = TLS(raw(packets[9][TLS]), tls_session=client_finished.tls_session.mirror())

In [ ]:
# Display the HTTP GET query
http_query = TLS(raw(packets[11][TLS]), tls_session=server_finished.tls_session.mirror())
http_query.show()

In [ ]:
# Display the answer containing the secret
http_response = TLS(raw(packets[13][TLS]), tls_session=http_query.tls_session.mirror())
http_response.show()